In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import data_loader
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from statsmodels.api import OLS
from xgboost import XGBRegressor

In [2]:
df = data_loader.load_data()
df

avg_income  avg_prop_tax  \
statefip Date                                      
Alabama  2000-01-31   41917.384970    271.925118   
         2000-02-29   42751.868482    277.935297   
         2000-03-31   43586.351994    283.945476   
         2000-04-30   44420.835507    289.955656   
         2000-05-31   45255.319019    295.965835   
...                            ...           ...   
Wyoming  2024-09-30  104455.891107      0.000000   
         2024-10-31  104455.891107      0.000000   
         2024-11-30  104455.891107      0.000000   
         2024-12-31  104455.891107      0.000000   
         2025-01-31  104455.891107      0.000000   

                     proportion_American Indian/Aleut/Eskimo  \
statefip Date                                                  
Alabama  2000-01-31                                 0.000000   
         2000-02-29                                 0.000387   
         2000-03-31                                 0.000775   
         2000-04-30                                 0.001162   
         2000-05-31                                 0.001549   
...                                                      ...   
Wyoming  2024-09-30                                 0.008195   
         2024-10-31                                 0.008195   
         2024-11-30                                 0.008195   
         2024-12-31                                 0.008195   
         2025-01-31                                 0.008195   

                     proportion_Asian only  proportion_Black  \
statefip Date                                                  
Alabama  2000-01-31               0.000000          0.262985   
         2000-02-29               0.000000          0.255512   
         2000-03-31               0.000000          0.248038   
         2000-04-30               0.000000          0.240565   
         2000-05-31               0.000000          0.233091   
...                                    ...               ...   
Wyoming  2024-09-30               0.007245          0.007625   
         2024-10-31               0.007245          0.007625   
         2024-11-30               0.007245          0.007625   
         2024-12-31               0.007245          0.007625   
         2025-01-31               0.007245          0.007625   

                     proportion_White  total_population  Median Home Price  
statefip Date                                                               
Alabama  2000-01-31          0.737015      706842.48000       97080.984792  
         2000-02-29          0.743665      682034.58299       97250.037728  
         2000-03-31          0.750316      657226.68598       97363.306485  
         2000-04-30          0.756967      632418.78897       97617.062414  
         2000-05-31          0.763617      607610.89196       97900.085921  
...                               ...               ...                ...  
Wyoming  2024-09-30          0.961807       67792.43000      341148.869753  
         2024-10-31          0.961807       67792.43000      343849.922709  
         2024-11-30          0.961807       67792.43000      346044.040121  
         2024-12-31          0.961807       67792.43000      348280.758951  
         2025-01-31          0.961807       67792.43000      350224.417540  

[15351 rows x 8 columns]

In [4]:
X_cols = df.columns[1:-1]
y_col = df.columns[-1]


dates = df.reset_index()['Date'].unique()
months = pd.DataFrame(np.arange(0, dates.size), index=dates)

df['Month'] = np.tile(np.arange(0, dates.size), 51)

In [5]:
for group, data in df.groupby('statefip'):
    print(f"State: {group}")
    X = data[X_cols]
    X = sm.add_constant(X)
    X['Month'] = np.arange(dates.size)
    print(data[y_col].name)
    y = data[y_col]
    
    model = OLS(y, X).fit()
    print(model.summary())

    # Lag of 1 year back in time
    X['lag_1']= 0
    X['lag_1'][12:] = y.iloc[:-12].values
    model = OLS(y, X).fit()
    print(model.summary())

State: Alabama
Median Home Price
                            OLS Regression Results                            
Dep. Variable:      Median Home Price   R-squared:                       0.924
Model:                            OLS   Adj. R-squared:                  0.922
Method:                 Least Squares   F-statistic:                     507.4
Date:                Mon, 31 Mar 2025   Prob (F-statistic):          1.07e-159
Time:                        17:11:53   Log-Likelihood:                -3191.2
No. Observations:                 301   AIC:                             6398.
Df Residuals:                     293   BIC:                             6428.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------

/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_96761/1799046534.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['lag_1'][12:] = y.iloc[:-12].values
/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_96761/1799046534.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['lag_1'][12:] = y.iloc[:-12].values
/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_96761/1799046534.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

                            OLS Regression Results                            
Dep. Variable:      Median Home Price   R-squared:                       0.844
Model:                            OLS   Adj. R-squared:                  0.840
Method:                 Least Squares   F-statistic:                     226.5
Date:                Mon, 31 Mar 2025   Prob (F-statistic):          3.18e-114
Time:                        17:11:53   Log-Likelihood:                -3497.5
No. Observations:                 301   AIC:                             7011.
Df Residuals:                     293   BIC:                             7041.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------

/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_96761/1799046534.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['lag_1'][12:] = y.iloc[:-12].values
/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_96761/1799046534.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['lag_1'][12:] = y.iloc[:-12].values
/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_96761/1799046534.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

                            OLS Regression Results                            
Dep. Variable:      Median Home Price   R-squared:                       0.926
Model:                            OLS   Adj. R-squared:                  0.925
Method:                 Least Squares   F-statistic:                     526.3
Date:                Mon, 31 Mar 2025   Prob (F-statistic):          7.65e-162
Time:                        17:11:53   Log-Likelihood:                -3559.2
No. Observations:                 301   AIC:                             7134.
Df Residuals:                     293   BIC:                             7164.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------

/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_96761/1799046534.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['lag_1'][12:] = y.iloc[:-12].values
/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_96761/1799046534.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['lag_1'][12:] = y.iloc[:-12].values
/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_96761/1799046534.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

In [51]:
for group, data in df.groupby('statefip'):
    print(f"State: {group}")
    X = data[X_cols]
    X = sm.add_constant(X)
    print(data[y_col].name)
    y = np.log(data[y_col]).rename(f"log({data[y_col].name})")
    
    model = OLS(y, X).fit()
    print(model.summary())

    # Lag of 1 year back in time
    X['lag_1']= 0
    X['lag_1'][12:] = y.iloc[:-12].values
    model = OLS(y, X).fit()
    print(model.summary())

State: Alabama
Median Home Price
                              OLS Regression Results                              
Dep. Variable:     log(Median Home Price)   R-squared:                       0.568
Model:                                OLS   Adj. R-squared:                  0.560
Method:                     Least Squares   F-statistic:                     64.52
Date:                    Sat, 22 Mar 2025   Prob (F-statistic):           8.26e-51
Time:                            14:30:48   Log-Likelihood:                 145.05
No. Observations:                     301   AIC:                            -276.1
Df Residuals:                         294   BIC:                            -250.2
Df Model:                               6                                         
Covariance Type:                nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------

/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_48513/2499356049.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['lag_1'][12:] = y.iloc[:-12].values
/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_48513/2499356049.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['lag_1'][12:] = y.iloc[:-12].values
/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_48513/2499356049.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

                              OLS Regression Results                              
Dep. Variable:     log(Median Home Price)   R-squared:                       0.574
Model:                                OLS   Adj. R-squared:                  0.565
Method:                     Least Squares   F-statistic:                     65.96
Date:                    Sat, 22 Mar 2025   Prob (F-statistic):           1.33e-51
Time:                            14:30:48   Log-Likelihood:                 109.32
No. Observations:                     301   AIC:                            -204.6
Df Residuals:                         294   BIC:                            -178.7
Df Model:                               6                                         
Covariance Type:                nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------

/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_48513/2499356049.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['lag_1'][12:] = y.iloc[:-12].values
/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_48513/2499356049.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['lag_1'][12:] = y.iloc[:-12].values
/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_48513/2499356049.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

                              OLS Regression Results                              
Dep. Variable:     log(Median Home Price)   R-squared:                       0.840
Model:                                OLS   Adj. R-squared:                  0.836
Method:                     Least Squares   F-statistic:                     219.2
Date:                    Sat, 22 Mar 2025   Prob (F-statistic):          1.84e-112
Time:                            14:30:48   Log-Likelihood:                 162.25
No. Observations:                     301   AIC:                            -308.5
Df Residuals:                         293   BIC:                            -278.8
Df Model:                               7                                         
Covariance Type:                nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------

/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_48513/2499356049.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['lag_1'][12:] = y.iloc[:-12].values
/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_48513/2499356049.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['lag_1'][12:] = y.iloc[:-12].values


In [6]:
params = []

for group, data in df.groupby('statefip'):
    # print(f"State: {group}")
    X = data[X_cols]
    X = sm.add_constant(X)
    # y = data[y_col]
    y = np.log(data[y_col]).rename(f"log({data[y_col].name})")
    
    model = OLS(y, X).fit()
    params.append(model.params)

print(pd.DataFrame(params).mean())
print(pd.DataFrame(params).std())

const                                      13.715031
avg_prop_tax                               -0.000197
proportion_American Indian/Aleut/Eskimo    -5.022432
proportion_Asian only                       8.859233
proportion_Black                           -0.008940
proportion_White                           -1.837985
total_population                            0.000002
dtype: float64
const                                      13.967222
avg_prop_tax                                0.000158
proportion_American Indian/Aleut/Eskimo    21.216873
proportion_Asian only                      22.277926
proportion_Black                           15.663762
proportion_White                           14.365178
total_population                            0.000004
dtype: float64


In [7]:
X = df[X_cols]
y = df[y_col]
X = sm.add_constant(X)

model = OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Median Home Price   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.304
Method:                 Least Squares   F-statistic:                     1118.
Date:                Mon, 31 Mar 2025   Prob (F-statistic):               0.00
Time:                        17:13:20   Log-Likelihood:            -1.9746e+05
No. Observations:               15351   AIC:                         3.949e+05
Df Residuals:                   15344   BIC:                         3.950e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===========================================================================================================
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
const                                    -5.13e+04   2.51e+04     -2.041      0.041   -1.01e+05   -2042.621
avg_prop_tax                              -18.5419      0.757    -24.487      0.000     -20.026     -17.058
proportion_American Indian/Aleut/Eskimo  3.168e+05   5.22e+04      6.068      0.000    2.14e+05    4.19e+05
proportion_Asian only                    1.399e+06      4e+04     34.971      0.000    1.32e+06    1.48e+06
proportion_Black                         2.995e+05   2.65e+04     11.286      0.000    2.47e+05    3.52e+05
proportion_White                         2.566e+05   2.55e+04     10.080      0.000    2.07e+05    3.06e+05
total_population                            0.0077      0.001      7.082      0.000       0.006       0.010
==============================================================================
Omnibus:                     2713.810   Durbin-Watson:                   0.020
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6257.662
Skew:                           1.012   Prob(JB):                         0.00
Kurtosis:                       5.384   Cond. No.                     9.39e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.39e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""